In [1]:
from dataloader import TFDNA_ds
import torch
import os
import helper_classes as hp
import numpy as np
import random
import matplotlib.pyplot as plt
import torch.nn as nn

In [2]:
n_layers = 5
d_model = 80
n_heads = 1
dropout_enc = 0.1
window_size = 300
dropout_emb = True
slide_step = 200
n_epochs = 10
figs = []
batch_size = 20

In [3]:
#allows the access to the files
cwd = os.getcwd()
os.chdir('/home/labs/antebilab/naamab_lab/Benny/Deep_learning_project/')

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
TF_path = 'Amino_acid_data/final_tf_data.h5'
DNA_path = 'signal_159_TFs'
data = TFDNA_ds(TF_path=TF_path, DNA_path=DNA_path, seq_length=window_size, sliding_window_step=slide_step)

# N_train = int(len(data)*10e-5)
# training_dataset = torch.utils.data.Subset(data,range(N_train))
# validation_dataset = torch.utils.data.Subset(data,range(N_train,len(data)))

N_train = (10 ** 4) * 5
N_valid = int(N_train / 10)
training_dataset = torch.utils.data.Subset(data,range(N_train))
validation_dataset = torch.utils.data.Subset(data,range(N_train, N_train + N_valid))

In [6]:
from model import EncoderNet, FCN

In [7]:
# net = EncoderNet(n_layers=n_layers, d_model=d_model, n_heads=n_heads, dropout_enc=dropout_enc, window_size=window_size, dropout_emb=dropout_emb)

In [17]:
input_size = 1502 + 1 + window_size
N_hidden = [input_size * 2, input_size * 4, input_size * 4, input_size * 2]
net = FCN(N_input=input_size, N_hidden=N_hidden, N_output=window_size, dropout=True)

In [18]:
net

FCN(
  (node_network): Sequential(
    (0): Linear(in_features=1803, out_features=3606, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(3606, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=3606, out_features=7212, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(7212, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=7212, out_features=7212, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(7212, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.1, inplace=False)
    (12): Linear(in_features=7212, out_features=3606, bias=True)
    (13): ReLU()
    (14): Linear(in_features=3606, out_features=300, bias=True)
  )
)

In [19]:
N_GPUs = torch.cuda.device_count()
if N_GPUs > 1:
    net = nn.DataParallel(net)
    batch_size = batch_size * N_GPUs
print(N_GPUs)

1


In [1]:
from torch.utils.data import Dataset, DataLoader

data_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

NameError: name 'training_dataset' is not defined

In [21]:
for batched_e,l in data_loader:
    break
batched_e.shape

torch.Size([20, 1803, 26])

In [22]:
device = torch.device('cuda')
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('cuda')
print(device)
net.to(device)

cuda
cuda:0


FCN(
  (node_network): Sequential(
    (0): Linear(in_features=1803, out_features=3606, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(3606, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=3606, out_features=7212, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(7212, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=7212, out_features=7212, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(7212, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.1, inplace=False)
    (12): Linear(in_features=7212, out_features=3606, bias=True)
    (13): ReLU()
    (14): Linear(in_features=3606, out_features=300, bias=True)
  )
)

In [23]:
import torch.nn as nn
import torch.optim as optim

loss_func = hp.Custom_MSE_Loss(c=4)
optimizer = optim.Adam(net.parameters(), lr=0.05) 

In [24]:
def compute_accuracy_and_loss(dataloader, net, valid=False):
    total = 0
    correct = 0
    loss = 0
    
    rnd_b = random.randint(1, int(N_valid/batch_size))
    rnd_i = random.randint(1, batch_size)
    
    if torch.cuda.is_available():
        net.cuda()
    net.eval()
    
    n_batches = 0
    with torch.no_grad():
        for batched_e, l in dataloader:
            n_batches += 1

            if torch.cuda.is_available():
                batched_e = batched_e.to(torch.device('cuda'))
                l = l.cuda()
            pred = net(batched_e)

            loss += loss_func(pred, l).item()
            
            if valid and n_batches == rnd_b:
                figs.append((pred[rnd_i], l[rnd_i], n_batches, rnd_i))

    loss = loss / n_batches      
    return loss

In [15]:
pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
from tqdm.notebook import tqdm

best_loss = np.inf

training_loss_vs_epoch = []
validation_loss_vs_epoch = []

# training_acc_vs_epoch = []
# validation_acc_vs_epoch = []

pbar = tqdm( range(n_epochs) )
os.chdir(cwd)

for epoch in pbar:
    
    if len(validation_loss_vs_epoch) > 1:
        pbar.set_description('val loss:'+'{0:.5f}'.format(validation_loss_vs_epoch[-1])+', train loss:'+'{0:.5f}'.format(training_loss_vs_epoch[-1]))
       
    net.train() # put the net into "training mode"
    for batched_e, l in data_loader:
        if torch.cuda.is_available():
            batched_e = batched_e.to(torch.device('cuda'))
            l = l.cuda()
            
        optimizer.zero_grad()
        pred = net(batched_e)
        loss = loss_func(pred, l)
        loss.backward()
        optimizer.step()
    
    net.eval() #put the net into evaluation mode
    train_loss = compute_accuracy_and_loss(data_loader, net)
    valid_loss =  compute_accuracy_and_loss(validation_data_loader, net, valid=True)
    print(valid_loss)
         
    training_loss_vs_epoch.append(train_loss)    
    # training_acc_vs_epoch.append( train_acc)
    # validation_acc_vs_epoch.append(valid_acc)
    validation_loss_vs_epoch.append(valid_loss)

    #keep model with best loss
    if valid_loss < best_loss:
        torch.save(net.state_dict(), 'trained_model.pt')
        best_loss = valid_loss

RuntimeError: mat1 and mat2 shapes cannot be multiplied (36060x26 and 1803x3606)

In [ ]:
print(training_loss_vs_epoch)

In [ ]:
print(validation_loss_vs_epoch)

In [ ]:
plt.plot(training_loss_vs_epoch,label='training')
plt.plot(validation_loss_vs_epoch,label='validation')
plt.title('Loss')
plt.legend()
plt.show()

In [ ]:
for e in figs:
    pred = e[0].numpy(force=True)
    label = e[1].numpy(force=True)
    batch = e[2]
    ind = e[3] 
    plt.plot(pred, label='pred')
    plt.plot(label, label='label')
    plt.title('batch number ' + str(batch) + ', example number ' + str(ind))
    plt.legend()
    plt.show()